# Image-Classifier-TensorFlow-Demo

In this demonstration I will train an image classifier using the PyTorch Framework to classify 102 species of flowers. For training I will be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories. You can see a few examples below.

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on the dataset
* Use the trained classifier to predict image content

In [3]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import time
import json

# Use GPU if it's available, else use cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Run autotuner to select kernel with best peroformance
torch.backends.cudnn.benchmark = True

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Load the data

Here we'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The dataset is split into three parts, training, validation, and testing. You can [download the dataset here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). For the training, we'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. We'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this we don't want any scaling or rotation transformations, but we'll need to resize then crop the images to the appropriate size.

The pre-trained networks we'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets we'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [1]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [4]:
# Define transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomResizedCrop(224, scale=(0.5, 1.5)),
                                       transforms.ColorJitter(brightness=0.03, hue=0.03),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

valid_test_transforms = transforms.Compose([transforms.Resize(224), 
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], 
                                                                 [0.229, 0.224, 0.225])])

# Load the datasets with ImageFolder
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)

valid_data = datasets.ImageFolder(data_dir + '/valid', transform=valid_test_transforms)

test_data = datasets.ImageFolder(data_dir + '/test', transform=valid_test_transforms)

# Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True, num_workers=6, pin_memory=True)

validloader = torch.utils.data.DataLoader(valid_data, batch_size=64, shuffle=False, num_workers=6, pin_memory=True)

testloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False, num_workers=6, pin_memory=True)

### Label mapping

We'll need to load in a mapping from category label to category name. We can find this in the file `cat_to_name.json`. It's a JSON object which we can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give us a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [5]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the model and classifier

Now that the data is ready, it's time to build and train the classifier. We will be using one of the pretrained models from `torchvision.models` to get the image features. Many pretrained classification models are available and can be found [here](https://pytorch.org/vision/stable/models.html). We will need to:

* Load a pretrained network and freeze the models weights
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

When training we will only be updating the weights of the new classifier we build while using the weights of the pretrained model to detect image features. We will then try some hyperparameter combinations (learning rate, units in the classifier, epochs, etc) to see if we can increase the model validation accuracy.

First I will define some functions and a class to help build the classifier and train the model.

In [7]:
# Load pretrained weights
weights='IMAGENET1K_V1'

# Load model using pretrained weights
model = models.maxvit_t(weights=weights)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

In [8]:
class Network(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_p=0.5):
        ''' Builds a feedforward network with arbitrary hidden layers.
        
            Arguments
            ---------
            input_size: integer, size of the input layer
            output_size: integer, size of the output layer
            hidden_layers: list of integers, the sizes of the hidden layers
            drop_p: float, dropout probability
        '''
        super().__init__()
        
        self.drop_p = drop_p
        
        # Input to a hidden layer
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0])])
        
        # Add a variable number of more hidden layers
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])
        
        self.output = nn.Linear(hidden_layers[-1], output_size)
        
    def forward(self, x):
        ''' Forward pass through the network, returns the output logits '''
        
        for each in self.hidden_layers:
            x = F.relu(each(x))
            x = F.dropout(x, self.drop_p)
        x = self.output(x)
        
        return F.log_softmax(x, dim=1)